# Kaggle Competition: Santander Customer Satisfaction 

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import train_test_split
import xgboost as xgb

%pylab inline

pd.options.display.max_columns=400 #permite visualizar 400 colunas

Populating the interactive namespace from numpy and matplotlib


### Leitura e tratamento dos dados:
* Seguindo uma dica do fórum da competição: "Replace -999999 in var3 column with most common value '2': https://www.kaggle.com/cast42/santander-customer-satisfaction/debugging-var3-999999"

In [4]:
%%time
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df.replace(-999999, 2, inplace=True)
df_test.replace(-999999, 2, inplace=True)

print df.shape

(76020, 371)
CPU times: user 10.6 s, sys: 1.5 s, total: 12.1 s
Wall time: 17.5 s


In [34]:
#Visualização da tabela
df.head(2)

#371 colunas!!!

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var5_0,ind_var5,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13,ind_var14_0,ind_var24_0,ind_var24,ind_var25_cte,ind_var26_cte,ind_var26,ind_var25,ind_var30,ind_var37_cte,ind_var37,ind_var39_0,ind_var40_0,ind_var41_0,num_var1_0,num_var1,num_var4,num_var5_0,num_var5,num_var8_0,num_var8,num_var12_0,num_var12,num_var13_0,num_var13_corto_0,num_var13_corto,num_var13_largo_0,num_var13_largo,num_var13,num_var14_0,num_var14,num_var17_0,num_var17,num_var20_0,num_var20,num_var24_0,num_var24,num_var26,num_var25,num_op_var40_hace2,num_op_var40_hace3,num_op_var40_ult1,num_op_var40_ult3,num_op_var41_hace2,num_op_var41_hace3,num_op_var41_ult1,num_op_var41_ult3,num_op_var39_hace2,num_op_var39_hace3,num_op_var39_ult1,num_op_var39_ult3,num_var30_0,num_var30,num_var31_0,num_var31,num_var32,num_var33_0,num_var35,num_var37_med_ult2,num_var37,num_var39_0,num_var40_0,num_var41_0,num_var39,num_var42_0,num_var42,num_var44_0,num_var44,saldo_var1,saldo_var5,saldo_var8,saldo_var12,saldo_var13_corto,saldo_var13_largo,saldo_var13,saldo_var14,saldo_var17,saldo_var18,saldo_var20,saldo_var24,saldo_var26,saldo_var25,saldo_var29,saldo_var30,saldo_var31,saldo_var32,saldo_var33,saldo_var34,saldo_var37,saldo_var40,saldo_var42,saldo_var44,var36,delta_imp_amort_var18_1y3,delta_imp_amort_var34_1y3,delta_imp_aport_var13_1y3,delta_imp_aport_var17_1y3,delta_imp_aport_var33_1y3,delta_imp_compra_var44_1y3,delta_imp_venta_var44_1y3,delta_num_aport_var13_1y3,delta_num_aport_var17_1y3,delta_num_aport_var33_1y3,delta_num_compra_var44_1y3,delta_num_reemb_var13_1y3,delta_num_reemb_var17_1y3,delta_num_reemb_var33_1y3,delta_num_trasp_var17_in_1y3,delta_num_trasp_var17_out_1y3,delta_num_trasp_var33_in_1y3,delta_num_trasp_var33_out_1y3,delta_num_venta_var44_1y3,imp_amort_var18_ult1,imp_amort_var34_ult1,imp_aport_var13_hace3,imp_aport_var13_ult1,imp_aport_var17_hace3,imp_aport_var17_ult1,imp_aport_var33_hace3,imp_aport_var33_ult1,imp_var7_emit_ult1,imp_var7_recib_ult1,imp_compra_var44_hace3,imp_compra_var44_ult1,imp_reemb_var13_ult1,imp_reemb_var17_hace3,imp_reemb_var17_ult1,imp_reemb_var33_ult1,imp_var43_emit_ult1,imp_trans_var37_ult1,imp_trasp_var17_in_hace3,imp_trasp_var17_in_ult1,imp_trasp_var17_out_ult1,imp_trasp_var33_in_hace3,imp_trasp_var33_in_ult1,imp_trasp_var33_out_ult1,imp_venta_var44_hace3,imp_venta_var44_ult1,ind_var10_ult1,ind_var10cte_ult1,ind_var9_cte_ult1,ind_var9_ult1,ind_var43_emit_ult1,ind_var43_recib_ult1,var21,num_aport_var13_hace3,num_aport_var13_ult1,num_aport_var17_hace3,num_aport_var17_ult1,num_var7_recib_ult1,num_compra_var44_ult1,num_ent_var16_ult1,num_var22_hace2,num_var22_hace3,num_var22_ult1,num_var22_ult3,num_med_var22_ult3,num_med_var45_ult3,num_meses_var5_ult3,num_meses_var8_ult3,num_meses_var12_ult3,num_meses_var13_corto_ult3,num_meses_var13_largo_ult3,num_meses_var39_vig_ult3,num_op_var39_comer_ult1,num_op_var39_comer_ult3,num_op_var40_comer_ult1,num_op_var40_comer_ult3,num_op_var40_efect_ult1,num_op_var40_efect_ult3,num_op_var41_comer_ult1,num_op_var41_comer_ult3,num_op_var41_efect_ult1,num_op_var41_efect_ult3,num_op_var39_efect_ult1,num_op_var39_efect_ult3,num_sal_var16_ult1,num_var43_emit_ult1,num_var43_recib_ult1,num_trasp_var11_ult1,num_venta_var44_ult1,num_var45_hace2,num_var45_hace3,num_var45_ult1,num_var45_ult3,saldo_medio_var5_hace2,saldo_medio_var5_hace3,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_v

### Feature selection
Como a base de dados de treino contém 76020 linhas e 371 colunas, precisamos selecionar quais colunas realmente contribuem com informação útil para detectar consumidores insatisfeitos. Então, abaixo removo colunas praticamente constantes (desvio padrão < 0.1) e colunas repetidas.

In [5]:
%%time

cols = []
for col in df.columns:
    if df[col].std() < 0.1:
        cols.append(col)

duplicados = []
for col in df.columns:
    for col2 in df.columns.difference([col] + duplicados):
        if (df[col] == df[col2]).all():
            duplicados.append(col)
            
cols+=duplicados
for col in pd.Series(cols).unique():
    if col == 'TARGET':
        continue
    try:
        del df_test[col]
        del df[col]
    except:
        print 'couldnt delete the row:' + str(col)

print pd.Series(cols).unique().shape
print df.shape, df_test.shape

(114,)
(76020, 257) (75818, 256)
CPU times: user 3min 42s, sys: 10.6 s, total: 3min 53s
Wall time: 6min 55s


##### Das 371 colunas iniciais, restam apenas 256 colunas.
A base de treino tem uma coluna a mais em relação à base de teste, que é a coluna 'Target'. Coluna que indica se o consumidor está insatisfeito ou não.

In [49]:
#Checando informacao sobre o Target. 

df.TARGET.value_counts()

#Apenas 3008/76020 = 0.039 = 3,9% dos dados de treino indicam consumidores insatisfeitos

0    73012
1     3008
Name: TARGET, dtype: int64

#### Agora, como ainda temos 256 colunas, eu as seleciono ainda mais com o modelo de classificação GradientBoostingClassifier. Ele atribui uma importância para cada coluna e o SelectFromModel as filtra. Qualquer outro modelo que atribui importância às features pode ser utilizado.

* GradientBoostingClassifier: http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
* SelectFromModel: http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html

In [37]:
%%time

#remove ID and TARGET from features
X = df[df.columns.difference(['ID','TARGET'])]
X_subm = df_test[df_test.columns.difference(['ID'])]

#get target
Y = df.TARGET

#feature selector
#min_samples_split=50 to avoid overfit
clf = GradientBoostingClassifier(verbose=1, min_samples_split=50)
clf.fit(X,Y)

#select features using default values and transform
model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X)
X_newsubm = model.transform(X_subm)

      Iter       Train Loss   Remaining Time 
         1           0.3197            3.14m
         2           0.3111            3.04m
         3           0.3049            2.88m
         4           0.3000            2.80m
         5           0.2960            2.74m
         6           0.2930            2.69m
         7           0.2908            2.62m
         8           0.2884            2.58m
         9           0.2861            2.54m
        10           0.2844            2.51m
        20           0.2742            2.23m
        30           0.2698            1.90m
        40           0.2673            1.58m
        50           0.2654            1.30m
        60           0.2644            1.02m
        70           0.2631           45.36s
        80           0.2623           30.19s
        90           0.2615           15.00s
       100           0.2607            0.00s
CPU times: user 2min 20s, sys: 7.85 s, total: 2min 28s
Wall time: 2min 29s


In [8]:
#train test split: divide em base de treino e teste para estimar score
X_train, X_test, Y_train, Y_test = train_test_split(X_new, Y)
print X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

(57015, 47) (19005, 47) (57015,) (19005,)


#### Agora restam apenas 47 colunas que realmente importam para o nosso problema!



### Parameter selection
#### * Abaixo utilizo o método GridSearchCV para encontrar os melhores parâmetros para o meu modelo de predição. O XGBoostClassifier;

#### * Observe que utilizo o método de scoring 'roc_auc', que é a métrica utilizada pela competição;

* GridSearchCV: http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html#sklearn.grid_search.GridSearchCV
* Métodos de scoring: http://scikit-learn.org/stable/modules/model_evaluation.html
* XGBoost: https://xgboost.readthedocs.io/en/latest/

#### "More than half of the winning solutions in machine learning challenges hosted at Kaggle adopt XGBoost."[http://www.kdnuggets.com/2016/03/xgboost-implementing-winningest-kaggle-algorithm-spark-flink.html]

In [28]:
#training
%%time
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold

parameter_grid = {
    'max_depth': range(2,7),
    'min_child_weight': range(1,7),
    'n_estimators': [100, 150, 200]
    
}
cross_validation = StratifiedKFold(Y_train, n_folds=10)

xg = xgb.XGBClassifier()
grid_search = GridSearchCV(xg, param_grid = parameter_grid, cv=cross_validation, scoring = 'roc_auc')
grid_search.fit(X_train, Y_train)


GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[0 0 ..., 0 0], n_folds=10, shuffle=False, random_state=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [100, 150, 200], 'max_depth': [2, 3, 4, 5, 6], 'min_child_weight': [1, 2, 3, 4, 5, 6]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [29]:
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

Best score: 0.841288228036
Best parameters: {'n_estimators': 100, 'max_depth': 4, 'min_child_weight': 4}


### Prediction 
* Melhores parâmetros selecionados acima geraram um bom score. Então, abaixo treino e faço a predição da base de teste

In [40]:
%%time
#submission

xg_new = grid_search.best_estimator_
xg_new.fit(X_new, Y)

results = xg_new.predict_proba(X_newsubm)

CPU times: user 12.9 s, sys: 114 ms, total: 13 s
Wall time: 13.1 s


#### * Criação do arquivo de submissão do Kaggle

In [31]:
%%time
subm = pd.DataFrame(columns=['ID', 'TARGET'])
subm.ID = df_test.ID
subm.TARGET = results[:, 1]
subm.to_csv('submission.csv', index=False)

print subm.shape #precisa ter 75818, 2

(75818, 2)
CPU times: user 131 ms, sys: 11.4 ms, total: 143 ms
Wall time: 143 ms
